In [ ]:
################################################################################################
# basic Python codes to read TROPOMI NO2 Level 2 files

# set up working environment
import os
import glob
import numpy  as np
import pandas as pd
import netCDF4 as nc   # don't know why "xarray" doesn't work for TropOMI
%matplotlib inline
import matplotlib.pyplot as plt
from gamap_colormap import WhGrYlRd
from mpl_toolkits.basemap import Basemap

In [ ]:
################################################################################################
# set working directory and load TROPOMI NO2 files
os.chdir("/rds/projects/2018/maraisea-glu-01/Study/Research_Data/TROPOMI/TROPOMI_NO2/")
TropOMI_files = sorted(glob.glob('*.nc')) 
TropOMI_files

In [ ]:
################################################################################################
# open one sample TROPOMI NO2 file
test = nc.Dataset(TropOMI_files[20], "r", format="NETCDF4")
print(test)

# there are no dimensions(sizes) or variables(dimensions) at this level
# "root group" describes the data
# "PRODUCT" and "METADATA" groups listed at the bottom contain the variables

In [ ]:
# "PRODUCT" group contains info about "dimensions(sizes) + variables(dimensions) + group(SUPPORT_DATA) + more subgroups"

# open "PRODUCT" group
test.groups['PRODUCT']         

# list details of all variables within "PRODUCT" group
test.groups['PRODUCT'].variables                           

# extract the variable "latitude"
test.groups['PRODUCT'].variables['latitude']

# open "SUPPORT_DATA" group and its subgroups
# notice the subgroups are listed at the bottom unless there is no more
test.groups['PRODUCT']['SUPPORT_DATA']                    
test.groups['PRODUCT']['SUPPORT_DATA']['GEOLOCATIONS']
test.groups['PRODUCT']['SUPPORT_DATA']['DETAILED_RESULTS']
test.groups['PRODUCT']['SUPPORT_DATA']['INPUT_DATA']

# extract the variable in the subgroup
test.groups['PRODUCT']['SUPPORT_DATA']['INPUT_DATA'].variables['eastward_wind']

In [ ]:
# get variables needed
lat  = test.groups['PRODUCT'].variables['latitude']
lon  = test.groups['PRODUCT'].variables['longitude']
Flag = test.groups['PRODUCT'].variables['qa_value']
Pre  = test.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision']
NO2  = test.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column']
wind_east  = test.groups['PRODUCT']['SUPPORT_DATA']['INPUT_DATA'].variables['eastward_wind']
wind_north = test.groups['PRODUCT']['SUPPORT_DATA']['INPUT_DATA'].variables['northward_wind']

# get NO2 attributes
Fill_value = NO2._FillValue
Unit_convert = NO2.multiplication_factor_to_convert_to_molecules_percm2

In [ ]:
# get the values from variables
lat  = test.groups['PRODUCT'].variables['latitude'][0][:][:]
lon  = test.groups['PRODUCT'].variables['longitude'][0][:][:]
Flag = test.groups['PRODUCT'].variables['qa_value'][0][:][:]
Pre  = test.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][0][:][:]

# get the data as an array and mask fill/missing values
NO2  = np.array(test.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0][:][:])
NO2[NO2==Fill_value]=np.nan

# convert unit for TROPOMI NO2
NO2 = NO2*Unit_convert 

# the NO2 range 
print(np.ma.masked_invalid(NO2).min(),np.ma.masked_invalid(NO2).max())

In [ ]:
# Date of this observation
date = test.time_reference.split("T")[0]

# convert date to weekday (0-6 Monday-Sunday)
import datetime
year,month,day = (int(x) for x in date.split('-'))    
weekday = datetime.date(year, month, day).weekday()

In [ ]:
# make the plot without any processing 
# can compare your plot with NASA Panapoly

# set up the figure
plt.figure(figsize=(25,20))
data = np.ma.masked_array(NO2, np.isnan(NO2))

# set map projection/resolution/domain
m = Basemap(projection='cyl', resolution='l',llcrnrlat= -90, urcrnrlat = 90,llcrnrlon=-180, urcrnrlon = 180)

# add coastlines
m.drawcoastlines(linewidth=0.5)

# add map grids and lat-lon labels
m.drawparallels(np.arange(-90., 120., 30.), labels=[1, 0, 0, 0]) 
m.drawmeridians(np.arange(-180, 180., 45.), labels=[0, 0, 0, 1]) 

# plot TROPOMI NO2 values on map
m.pcolormesh(lon, lat, data, latlon=True, cmap='jet',vmin = 0,vmax = 0.00003*Unit_convert)

# add plot title

product   = 'TROPOMI NO2'
map_title = 'Sample plot'

plt.title('{0}\n{1}'.format(product,map_title), fontsize = 40) 
plt.show()